# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [1]:
from openai import OpenAI

client = OpenAI(base_url="https://burn.hair/v1",api_key="sk-BwhV3ToVt5uUdcE5Af24D16c0016440cB03e9c817c68Fc83")

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展现了一片迷人的自然景观，其中包括一条木制的栈道穿越在一片翠绿的高草中。栈道直通远方，引人深入探索这片未被开发的自然区域。画面中，晴朗的蓝天与飘动的白云为背景，增添了一种宁静和平和的气氛。四周的植被非常丰富，显示出多样的绿色调和一些花草树木。整体来看，这是一幅充满生机与自然美的图像，适合强调自然保护和户外活动的重要性。', role='assistant', function_call=None, tool_calls=None))


In [3]:
response.choices[0].message.content

'这幅图展示了一个美丽的自然风景。图片中心是一条木质步道，伸展直入一片郁郁葱葱的草地中。步道两旁的草地郁郁葱葱，显示出生机盎然的绿色，草地中散布着一些低矮的灌木和树木。天空呈现出饱和的蓝色，布满了绵延的白云。\n\n这幅图的色彩鲜明，给人一种宁静平和的感觉，仿佛邀请观者走进这个自然的世界中探索和放松。步道提供了一条明确的路径，引领观者的目光向图片的深处延伸，同时也象征着探险与发现的旅程。整幅图的构图和色彩都非常和谐，是一副典型的自然美景摄影作品。'

### 封装成一个函数 query_image_description

In [4]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(base_url="https://burn.hair/v1")  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [5]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一个幽默的比较，展示了两只经过幻想编辑的狗。左边的狗被赋予了非常夸张的肌肉身体，看起来像一个健美运动员，上面的文字是“16岁的我，工作后的我”，下方写着“我刚起一大早也得，拿豪猪肛干，少年我活的太快，四处粘有肛门的味道”，显然是一种自嘲的幽默表达，夸张地描述了年轻时的活力和工作后的变化。右边的狗则显得更为普通，体型较肥胖，坐姿表现出疲惫感，下面的文字“好累好想卸，好想要泡面，厨艺也不行要，洗碗我拖拉，洗太碗，要栽人”，同样以一种幽默的方式表达了工作后个体的疲惫和对日常琐事的不满意。

总的来说，这是一副调侃人们在青少年时期与成年后状态对比的幽默图片，通过夸张的形象和搞笑的文本来表达这种对比和感受。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [4]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(base_url="https://api.xiaoai.plus/v1")  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [8]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这张图显示了1980年到2020年间美国、中国、日本和德国的国内生产总值（GDP）比较。横坐标代表年份，纵坐标代表GDP（以万亿美元计）。图中的线条分别用不同颜色表示不同国家的GDP趋势：

- 蓝色线代表美国，可以看出美国的GDP持续增长，从1980年的约3万亿美元增长到2020年的接近21万亿美元。
- 红色线代表中国，显示了自1980年代初期的低起点后，中国GDP的迅速增长，特别是从2000年以后增长非常显著，到2020年达到约14万亿美元。
- 紫色线代表日本，其GDP在1990年代初达到高峰后，增长放缓，之后基本保持稳定。
- 绿色线代表德国，GDP增长较为平稳，在2020年约为4万亿美元。

总体上，这张图展示了四个经济体在过去40年中的经济表现和增长趋势，其中美国和中国的经济增长最为显著。


#### 使用 GPT-4V 识别手写体笔记

![](./images/bruce_test.jpg)

In [5]:
content = query_base64_image_description("./images/bruce_test.jpg")
print(content)

图中显示的是一些红色手写文字。右上角写着“Bruce Feng”，而左下角写着“2017 OpenAI 团队”。这些文字可能是某个人的名字“Bruce Feng”以及相关的一些信息，即他（或者她）可能在2017年加入了OpenAI公司的团队。这似乎是对某个具体情况的记录或注释。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [10]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片展示的是一本笔记簿的一页，内容主要涉及到机器学习中的几种模型调整技术，特别是用于深度学习模型的参数调整和优化。这些笔记看起来是专注于NLP（自然语言处理）中的应用。下面是详细的内容解释：

1. **Prompt Tuning（提示调整）**: 提到了这是适用于小型模型的一种技术。具体来说，这种方法涉及修改输入数据的一部分，使其能够引导模型以特定的方式响应。这里X表示输入数据的嵌入表示向量，Y = WX' 公式显示了输出Y是通过X的变换后的结果。

2. **Prefix Tuning**: 这里提到了为输入X添加前缀W，并使用变换器（Transformer）架构处理这些数据。

3. **LoRA（Low-Rank Adaptation 低秩适应）**: 这种技术通过对权重矩阵W进行低秩修改ΔW，实现模型的快速适应。ΔW 通过两个更小的矩阵A和B进行参数化，这可以减少需要训练的参数数量，从而加快训练速度并降低内存需求。公式中，ΔW = AB，说明了这种低秩分解方式。

4. 文本中还提及了模型大小和内存需求，比如LoRA模型需求约为78GB，而Q-LoRA和LaMA-6513则需要较少的内存（例如LaMA-6513需要48GB）。

这张图片反映了一些先进的技术在处理大规模数据时如何通过节省计算资源来进行有效的模型优化。

![](./images/handwriting_1.jpg)

In [11]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片展示的是一个笔记本上手写的内容，主要涉及了自然语言处理、特别是变换模型（Transformers）相关的技术和方法。以下是对图片中内容的一些解释：

1. **Transformers 和 Benchmark** - 谈及了变换模型及其在一些标准测试中的表现。
2. **PEFT/SOTA 和 PBFT Methods** - 提到了可能是对变换模型进行前端训练（Pre-Finetuning）的一些最新的方法或最优的技术（State of the Art）。
3. **Prompt Tuning** 和 **Adapter** 方法 - 这些都是调整预训练语言模型以适应特定任务的技术。具体包括：Adapter (2019, Google)、Prefix (2021, Stanford)、Prompt (2021, Google)、P-Tuning V1 (2021) 等。
4. **LORA, QLORA, Adalora** 等新方法 - 这些可能是近期提出的新技术，用于改进模型的微调过程。
5. **MAM Adapters** - 可能指的是一种新的Adapter架构，用于在大规模语言模型中进行更有效的微调。

此外，还有提及到一些特定的大型模型，如GPT-X和LLMs（大型语言模型），以及一些可能的改进技术如插入形式（Insertion form）和对比函数（Comparison function）。这些笔记表明，作者在深入研究如何通过各种先进的技术来增强和优化语言模型的性能。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。